In [10]:
# Workspace problem with several narrow gaps

import torch
import torch.nn as nn

from functools import partial
import numpy as np
from io import StringIO
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree as KDTree
import sys
import time

sys.path.append('/home/oscar_palfelt/MSc_thesis/ompl/py-bindings')
from ompl import base as ob
from ompl import control as oc
from ompl import geometric as og

cuda = False
DEVICE = torch.device("cuda" if cuda else "cpu")

In [2]:
# neural network parameters
mb_size = 256 # mini batch dim
h_Q_dim = 512 # encoder dim
h_P_dim = 512 # decoder dim

# problem dimenc_dimsions
dim = 4 # (x, y, xdot, ydot)
dataElements = dim+3*3+2*dim # sample (4D), gap1 (2D, 1D orientation), gap2, gap3, init (4D), goal (4D)

z_dim = 2 # latent dim
X_dim = dim # samples dim

In [3]:
# read in data
# data from https://github.com/StanfordASL/LearnedSamplingDistributions
filename = '/home/oscar_palfelt/MSc_thesis/LearnedSamplingDistributions/narrowDataFile.txt'
data = np.genfromtxt(filename, delimiter=',', dtype='d', usecols=[0,1,3,4,6,7,8,9,10,11,12,13,14,15,16,18,19,21,22,24,25])
numEntries = data.shape[0]

In [ ]:
# change conditions to occupancy grid
# code from https://github.com/StanfordASL/LearnedSamplingDistributions

def isSampleFree(sample, obs):
    for o in list(range(0,obs.shape[0]//(2*dimW))): # python 2 -> 3: use list(), use //
        isFree = 0
        for d in range(0,sample.shape[0]):
            if (sample[d] < obs[2*dimW*o + d] or sample[d] > obs[2*dimW*o + d + dimW]):
                isFree = 1
                break
        if isFree == 0:
            return 0
    return 1

gridSize = 11
dimW = 3
plotOn = False;

# process data into occupancy grid
conditions = data[0:numEntries,dim:dataElements]
conditionsOcc = np.zeros([numEntries,gridSize*gridSize])
occGridSamples = np.zeros([gridSize*gridSize, 2])
gridPointsRange = np.linspace(0,1,num=gridSize)

idx = 0;
for i in gridPointsRange:
    for j in gridPointsRange:
        occGridSamples[idx,0] = i
        occGridSamples[idx,1] = j
        idx += 1;

for j in range(0,numEntries,1):
    dw = 0.1
    dimW = 3
    gap1 = conditions[j,0:3]
    gap2 = conditions[j,3:6]
    gap3 = conditions[j,6:9]

    obs1 = [0, gap1[1]-dw, -0.5,             gap1[0], gap1[1], 1.5]
    obs2 = [gap2[0]-dw, 0, -0.5,             gap2[0], gap2[1], 1.5];
    obs3 = [gap2[0]-dw, gap2[1]+dw, -0.5,    gap2[0], 1, 1.5];
    obs4 = [gap1[0]+dw, gap1[1]-dw, -0.5,    gap3[0], gap1[1], 1.5];
    obs5 = [gap3[0]+dw, gap1[1]-dw, -0.5,    1, gap1[1], 1.5];
    obs = np.concatenate((obs1, obs2, obs3, obs4, obs5), axis=0)
    
    if j % 5000 == 0:
        print('Iter: {}'.format(j))
        
    occGrid = np.zeros(gridSize*gridSize)
    for i in range(0,gridSize*gridSize):
        occGrid[i] = isSampleFree(occGridSamples[i,:],obs)
    conditionsOcc[j,:] = occGrid
    
cs = np.concatenate((data[0:numEntries,dim+3*dimW:dataElements], conditionsOcc), axis=1) # occ(11x11), init (4D), goal (4D)
c_dim = cs.shape[1]

In [5]:
# define pytorch networks
# based on https://github.com/Jackson-Kang/Pytorch-VAE-tutorial/blob/master/.ipynb_checkpoints/01_Variational_AutoEncoder-checkpoint.ipynb

class Encoder(nn.Module):
    
    def __init__(self, input_dim=X_dim+c_dim, hidden_dim=h_Q_dim, latent_dim=z_dim):
        super(Encoder, self).__init__()

        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )

        self.z_mu = nn.Linear(hidden_dim, latent_dim)
        self.z_logvar = nn.Linear(hidden_dim, latent_dim)
        
    def forward(self, x):

        seq = self.network(x)

        return self.z_mu(seq), self.z_logvar(seq)


class Decoder(nn.Module):
    def __init__(self, latent_dim=z_dim+c_dim, hidden_dim=h_P_dim, output_dim=X_dim):
        super(Decoder, self).__init__()

        self.network = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
        
    def forward(self, x):
        
        return self.network(x)


class NeuralNetwork(nn.Module):
    def __init__(self, Encoder, Decoder):
        super(NeuralNetwork, self).__init__()
        self.Encoder = Encoder
        self.Decoder = Decoder
    
    def reparameterization(self, mean, logvar):
        epsilon = torch.randn_like(mean).to(DEVICE)        # sampling epsilon        
        z = mean + torch.exp(0.5 * logvar) * epsilon       # reparameterization trick
        return z

    def forward(self, x, c, encode=True):
        if encode:
            z_mu, z_logvar = self.Encoder(torch.cat((x, c), dim=1))
            z = self.reparameterization(z_mu, z_logvar)

            y = self.Decoder(torch.cat((z, c), dim=1))
            
            return y, z_mu, z_logvar
        else:
            z = x
            y = self.Decoder(torch.cat((z, c), dim=1))    

            return y

network = torch.load('/home/oscar_palfelt/MSc_thesis/EECS_Degree_Project/learn_distributions/network_name.pt')

In [6]:
# define planning problem

def getThresholdPathLengthObj(si):
     obj = ob.PathLengthOptimizationObjective(si)
     obj.setCostThreshold(ob.Cost(2.0))
     return obj


class MyStateSampler(ob.StateSampler):
    def __init__(self, space):
         super(MyStateSampler, self).__init__(space)
         self.name_ = "my sampler"
         self.bounds = space.getBounds()

    def sampleUniform(self, state):
        if np.random.uniform() < p:
            rndState = y[np.random.randint(low=0, high=y.shape[0]-1), :]
            state.setYaw(float(int(rndState[3] > 0) * np.arccos(np.dot(rndState[2:4] / np.linalg.norm(rndState[2:4]), [1, 0])))) # this is wrong
        else:
            rndState = [np.random.uniform(low=self.bounds.low[0], high=self.bounds.high[0]),
                        np.random.uniform(low=self.bounds.low[1], high=self.bounds.high[1]),
                        np.random.uniform(low=-np.pi, high=np.pi)]
            state.setYaw(rndState[2])

        state.setX(float(rndState[0]))
        state.setY(float(rndState[1]))


def allocStateSampler(space):
    return MyStateSampler(space)


def isStateValid(spaceInformation, state):
    # perform collision checking or check if other constraints are
    # satisfied
    u = int(np.floor(state.getX() * occGrid.shape[1])) # right pointing image axis
    v = int(np.floor(occGrid.shape[0] * (1 - state.getY()))) # down pointing image axis

    if spaceInformation.satisfiesBounds(state):
        return occGrid[v,u] > 0


def problemDef():
    # construct the state space we are planning in
    space = ob.DubinsStateSpace(turningRadius=0.1)

    # set the bounds for the R^2 part of SE(2)
    bounds = ob.RealVectorBounds(2)
    bounds.setLow(0.001)
    bounds.setHigh(0.999)
    space.setBounds(bounds)

    # set state sampler
    space.setStateSamplerAllocator(ob.StateSamplerAllocator(allocStateSampler))

    # define a simple setup class
    ss = og.SimpleSetup(space)
    ss.setStateValidityChecker(ob.StateValidityCheckerFn( \
        partial(isStateValid, ss.getSpaceInformation())))

    si = ss.getSpaceInformation()
    planner = og.RRTstar(si)
    ss.setPlanner(planner)

    ss.getProblemDefinition().setOptimizationObjective(getThresholdPathLengthObj(si))
    
    return ss


def plan(planObj, initState, goalState):

    space = ob.DubinsStateSpace(turningRadius=0.1)
    
    start = ob.State(space)
    start().setX(initState[0])
    start().setY(initState[1])
    start().setYaw(initState[2])

    goal = ob.State(space)
    goal().setX(goalState[0])
    goal().setY(goalState[1])
    goal().setYaw(goalState[2])
    
    planObj.setStartAndGoalStates(start, goal, 0.1)

    timeTerminationCondition = ob.timedPlannerTerminationCondition(5.0)
    exactSolTerminationCondition = ob.exactSolnPlannerTerminationCondition(planObj.getProblemDefinition())

    planObj.solve(ob.plannerOrTerminationCondition(exactSolTerminationCondition, timeTerminationCondition))

In [ ]:
# compare performance of planning w/ and w/out the learned distributions

nTests = 1

nSamples = 2000 # number of samples to draw in latent space

planObj = problemDef()
minCostThreshold = 0.0 # set minimumcost threshold to only attempt complex planning scenarios

nTotIterations = np.array([0, 0])
nCompletedTests = 0

plotPaths = True
while nCompletedTests <= nTests - 1:

    planIdx = np.random.randint(0,numEntries-1); # chose a random test scenario

    c_sample_seed = cs[planIdx,:]
    c_sample = torch.from_numpy(np.repeat([c_sample_seed],nSamples,axis=0)).float().to(DEVICE)

    # generate state space samples from the CVAE
    y = network(torch.randn(nSamples, z_dim).to(DEVICE), c_sample, encode=False).cpu().detach().numpy()

    start = data[planIdx, 13:16]
    goal = data[planIdx, 17:20]
    # the start/goal yaw is constant 0 which might not be suitable. For improvement, the initial/goal yaw should be determined
        
    occGrid = np.rot90(c_sample_seed[-gridSize*gridSize:].reshape(gridSize, gridSize))

    nIterations = np.array([0, 0])
    for i, p in enumerate([0., 0.6]): # p is probability of using the NN during sampling
        planObj.clear()
        plan(planObj, initState=start, goalState=goal)

        if planObj.getProblemDefinition().hasExactSolution():
            if planObj.getPlanner().bestCost().value() > minCostThreshold:
                sol = np.loadtxt(StringIO(planObj.getProblemDefinition().getSolutionPath().printAsMatrix()))
                nIterations[i] += planObj.getPlanner().numIterations()
                
                if plotPaths:
                    fig1 = plt.figure(figsize=(10,6), dpi=80)
                    ax1 = fig1.add_subplot(111, aspect='equal')
                    ax1.imshow(occGrid, extent=[0, gridSize-1, 0, gridSize-1], cmap='gray')
                    ax1.scatter(y[:,0] * (gridSize - 1), y[:,1] * (gridSize - 1), color="green", s=70, alpha=0.1)
                    ax1.quiver(sol[:, 0] * (gridSize - 1), sol[:, 1] * (gridSize - 1), np.cos(sol[:,2]), np.sin(sol[:,2]), color="purple", scale=8.0, width=0.015, alpha=0.9)
                    ax1.scatter(start[0] * (gridSize - 1), start[1] * (gridSize - 1), color="red", s=250, edgecolors='black') # init
                    ax1.scatter(goal[0] * (gridSize - 1), goal[1] * (gridSize - 1), color="blue", s=250, edgecolors='black') # goal
                    plt.show()
            else:
                nIterations = np.array([0, 0])
                nCompletedTests -= 1
                break 
        else:
            nIterations = np.array([0, 0])
            nCompletedTests -= 1
            break
    
    nTotIterations += nIterations
    nCompletedTests += 1


In [22]:
print("Average # of iterations: ", nTotIterations / nTests)

Average # of iterations:  [ 61. 330.]
